In [207]:
import warnings
warnings.filterwarnings("ignore")

In [208]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

food = pd.read_csv("/Users/viktorijakesaite/Desktop/CAMBRIDGE POSTDOC/UPF/data files/foodleveldietarydatauk_new.csv", encoding='latin1')

Exclude alcohol and supplements

In [209]:
categories= {'47A', '47B','48A', '48B', '48C', '49A', '49B', '49C', '49D', '49E', '54A', '54B', '54C', '54D', '54E', '54F', '54G', '54H', '54I', '54J', '54K', '54L', '54M', '54N', '54P'}
food_noalcnovit=food[~food['SubFoodGroupCode'].isin(categories)]

Check the categories within 'MainFoodGroupDesc'

In [210]:
print(np.unique (food['MainFoodGroupDesc']))
values, counts = np.unique(food['MainFoodGroupDesc'], return_counts=True)
print ('Values: ', values)

['1% Fat Milk' 'ARTIFICIAL SWEETENERS' 'BACON AND HAM'
 'BEEF VEAL AND DISHES' 'BEER LAGER CIDER & PERRY' 'BISCUITS'
 'BROWN GRANARY AND WHEATGERM BREAD' 'BUNS CAKES PASTRIES & FRUIT PIES'
 'BURGERS AND KEBABS' 'BUTTER' 'CHEESE' 'CHICKEN AND TURKEY DISHES'
 'CHIPS FRIED & ROAST POTATOES AND POTATO PRODUCTS'
 'CHOCOLATE CONFECTIONERY' 'COATED CHICKEN'
 'COMMERCIAL TODDLERS FOODS AND DRINKS' 'CRISPS AND SAVOURY SNACKS'
 'DIETARY SUPPLEMENTS' 'EGGS AND EGG DISHES' 'FRUIT' 'FRUIT JUICE'
 'HIGH FIBRE BREAKFAST CEREALS' 'ICE CREAM' 'LAMB AND DISHES'
 'LIVER & DISHES' 'LOW FAT SPREAD' 'MEAT PIES AND PASTRIES'
 'MISCELLANEOUS' 'NUTS AND SEEDS' 'OILY FISH' 'OTHER BREAD'
 'OTHER BREAKFAST CEREALS' 'OTHER MARGARINE FATS AND OILS'
 'OTHER MEAT AND MEAT PRODUCTS' 'OTHER MILK AND CREAM'
 'OTHER POTATOES POTATO SALADS & DISHES'
 'OTHER WHITE FISH SHELLFISH & FISH DISHES' 'PASTA RICE AND OTHER CEREALS'
 'PORK AND DISHES' 'PUDDINGS' 'PUFA MARGARINE & OILS' 'REDUCED FAT SPREAD'
 'SALAD AND OTHER RAW VEG

Check the food sample size of the original data

In [211]:
len (food)

1756207

Check the food sample size after removing supplements and alcohol

In [212]:
len (food_noalcnovit)

1730158

1.1	Classify HFSS foods using the UK Nutrient Databank based on the NPM 2004/5 version. 

We do not have veg and fruit purees in the NDNS dataset, so we first create these two categories

Create vegetable and fruit purees -- based on 'FoodName' variable

In [213]:
veg_pureed={'BROCOLLI SPEARS FRESH BOILED PUREED','CABBAGE-WINTER KALE FRESH BOILED (PUREED)',
            'CARROT OLD RAW PUREED','CARROTS OLD, FRESH, BOILED PUREED', 'CARROTS YOUNG FRESH BOILED PUREED', 'CARROTS YOUNG FRESH RAW PUREED',  
            'CAULIFLOWER-FRESH BOILED (PUREED)', 'CELERIAC FRESH BOILED (PUREED)', 'CELERY FRESH BOILED PUREED',  'CELERY, FRESH RAW (PUREED)',
            'COURGETTE-BOILED (PUREED)','GARLIC PUREE','GUACAMOLE PURCHASED', 'GUACAMOLE REDUCED FAT PURCHASED',
            'HARICOT BEANS, NO ADDED SALT, CANNED AND DRAINED, COOKED (PUREED)','HUMMUS, NOT CANNED', 'HUMMUS/HOUMOUS, LOW/REDUCED FAT',
            'LEEKS FRESH BOILED (PUREED)', 'LENTILS SPLIT BOILED (PUREED)', 'MUSHROOMS STEWED OR GRILLED (PUREED)',
            'ONIONS BOILED PUREED', 'PEAS FROZEN BOILED (PUREED) FF PROJECT', 'PEAS SPLIT DRIED BOILED (PUREED)', 
            'PEAS-CHICK, NO ADDED SALT OR SUGAR, CANNED AND DRAINED , COOKED (PUREED)', 'PEPPERS GREEN BOILED (PUREED)', 
            'PEPPERS YELLOW FRESH BOILED (PUREED)',  'PUMPKIN, PUREED, BOILED', 'SPINACH RAW NOT BABY SPINACH (PUREED)', 'SPINACH FRESH BOILED (PUREED)', 
            'SWEDE BOILED (PUREED)', 'SWEETCORN, CANNED, DRAINED, NON ADDED SUGAR OR SALT (PUREED) FS PROJECT', 'TURNIPS-BOILED (PUREED)'}

In [214]:
fruit_pureed={'GOJI BERRIES / WOLFBERRIES DRIED (PUREED)','MELON PUREE, HOMEMADE, 100% FRESH RAW FRUIT NAS','STRAWBERRIES STEWED WITHOUT SUGAR (PUREED)', 
              'PURE FRUIT PUREES NAS ANY FLAVOUR READY TO EAT (ADDED VIT C 0-10MG)', 
              'PURE FRUIT PUREES NAS ANY FLAVOUR READY TO EAT (ADDED VIT C 10-20MG)',
              'PURE FRUIT PUREES NAS ANY FLAVOUR READY TO EAT (ADDED VIT C OVER 20MG)', 'GRAPEFRUIT RAW, PUREED, FLESH ONLY NO PEEL OR PIPS', 
              'KIWI PUREE, HOMEMADE, 100% FRESH RAW FRUIT NAS', 'NECTARINE PUREE, HOMEMADE, 100% FRESH RAW FRUIT NAS', 
              'PASSION FRUIT RAW FLESH & SEEDS ONLY (PUREED)', 'RASPBERRY PUREE, HOMEMADE, 100% FRESH RAW FRUIT NAS'}

In [215]:
food_noalcnovit.loc[:, 'veg_pureed']  = 0
food_noalcnovit.loc[:, 'fruit_pureed']  = 0
food_noalcnovit.loc[food_noalcnovit['FoodName'].isin(veg_pureed), 'veg_pureed'] = food_noalcnovit.loc[food_noalcnovit['FoodName'].isin(veg_pureed), 'TotalGrams']
food_noalcnovit.loc[food_noalcnovit['FoodName'].isin(fruit_pureed), 'fruit_pureed'] = food_noalcnovit.loc[food_noalcnovit['FoodName'].isin(fruit_pureed), 'TotalGrams']

Calculate vegetable variable by adding up all the vegetables in the dataset

In [216]:
food_noalcnovit.loc [:, "vegetable"] = food_noalcnovit["Tomatoesg"]+ food_noalcnovit["Brassicaceaeg"]+food_noalcnovit["YellowRedGreeng"]+food_noalcnovit["Beansg"]+food_noalcnovit["OtherVegg"]

Calculate fruit, veg, and nuts per 100g. Here we also add fruit and veg purees (as suggested by the NPM documentation we multiply all purees and dried nuts by 2)

In [217]:
food_noalcnovit.loc [:,"fvn_per_100_g"] = (food_noalcnovit["Fruitg"] + food_noalcnovit["vegetable"]+ food_noalcnovit["Nutsg"]+ food_noalcnovit['TomatoPureeg']*2+ food_noalcnovit['veg_pureed']*2+ food_noalcnovit['fruit_pureed']*2+food_noalcnovit['DriedFruitg']*2)/(food_noalcnovit["TotalGrams"]+ food_noalcnovit['TomatoPureeg'] + food_noalcnovit['veg_pureed']+ food_noalcnovit['fruit_pureed']+ food_noalcnovit['DriedFruitg'])*100

Calculate the amount per 100g

In [218]:
food_noalcnovit.loc [:, "Fruitjuice_per_100_g"] = food_noalcnovit["FruitJuiceg"]/food_noalcnovit["TotalGrams"]*100

In [219]:
food_noalcnovit.loc [:, "Smoothies_per_100_g"] = food_noalcnovit["SmoothieFruitg"]/food_noalcnovit["TotalGrams"]*100

In [220]:
food_noalcnovit.loc [:, "EnergykJ_per_100_g"] = food_noalcnovit["EnergykJ"]/food_noalcnovit["TotalGrams"]*100

In [221]:
food_noalcnovit.loc [:, "Saturatedfattyacidsg_per_100_g"] = food_noalcnovit["Saturatedfattyacidsg"]/food_noalcnovit["TotalGrams"]*100

In [222]:
food_noalcnovit.loc [:, "Totalsugarsg_per_100_g"] = food_noalcnovit["Totalsugarsg"]/food_noalcnovit["TotalGrams"]*100

In [223]:
food_noalcnovit.loc [:, "Sodiummg_per_100_g"] = food_noalcnovit["Sodiummg"]/food_noalcnovit["TotalGrams"]*100

In [224]:
food_noalcnovit.loc [:, "AOACFibreg_per_100_g"] = food_noalcnovit["AOACFibreg"]/food_noalcnovit["TotalGrams"]*100

In [225]:
food_noalcnovit.loc [:, "Proteing_per_100_g"] = food_noalcnovit["Proteing"]/food_noalcnovit["TotalGrams"]*100

Explanation for the coding for nova are provided elsewhere: Colombet Z., O’Flaherty M., Chavez-Ugalde Y. NOVA classification of the National Diet and Nutrition Survey, waves 1 to 11 (2008/09 to 2018/19): GitHub; 2023 [Available from: https://github.com/zoecolombet/NOVA_NDNS_code.

Check the percent in each nova category

In [226]:
food_noalcnovit.NOVANumNEW_agreement.value_counts(normalize=True)*100

Unprocessed or minimally processed foods    49.459298
Ultra-processed food and drink products     36.210392
Processed culinary ingredients               8.208614
Processed foods                              6.121695
Name: NOVANumNEW_agreement, dtype: float64

Create a binary category: Ultra-processed food and drink products equals 1, and 0 otherwise (unprocessed or minimally processed foods, processed culinary ingredients, processed foods)

In [227]:
food_noalcnovit.loc[:, 'NOVA_UPF_NUPF_boolean'] = (food_noalcnovit['NOVANumNEW_agreement'] == 'Ultra-processed food and drink products')

Check the percentage of foods that fall into UPF and non-UPF

In [228]:
food_noalcnovit.NOVA_UPF_NUPF_boolean.value_counts(normalize=True)*100

False    63.789608
True     36.210392
Name: NOVA_UPF_NUPF_boolean, dtype: float64

Classify HFSS foods using NDNS food database based on the NPM 2004/5 version. This is based on the description provided in the documentation on the NPM. (Available at: https://www.gov.uk/government/publications/the-nutrient-profiling-model)

A POINTS

In [229]:
energy_points = {0: [-1, 335], 1: [335, 670], 2: [670, 1005], 3: [1005, 1340], 
                 4: [1340, 1675], 5: [1675, 2010], 6: [2010, 2345], 7: [2345, 2680],
                 8: [2680, 3015], 9: [3015, 3350], 10: [3350]}
food_noalcnovit.loc [:, 'energy_a_points'] = np.array(len(food_noalcnovit) * [0])
for points in energy_points:
    interval = energy_points[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['EnergykJ_per_100_g'] > interval[0]) & (food_noalcnovit['EnergykJ_per_100_g'] <= interval[1]),'energy_a_points'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['EnergykJ_per_100_g'] > interval[0]), 'energy_a_points'] = points

In [230]:
satfat_points = {0: [-1, 1], 1: [1, 2], 2: [2, 3], 3: [3, 4], 
                 4: [4, 5], 5: [5, 6], 6: [6, 7], 7: [7, 8],
                 8: [8, 9], 9: [9, 10], 10: [10]}
food_noalcnovit.loc[:, 'satfat_a_points'] = np.array(len(food_noalcnovit) * [0])
for points in satfat_points:
    interval = satfat_points[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['Saturatedfattyacidsg_per_100_g'] > interval[0]) & (food_noalcnovit['Saturatedfattyacidsg_per_100_g'] <= interval[1]),'satfat_a_points'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['Saturatedfattyacidsg_per_100_g'] > interval[0]), 'satfat_a_points'] = points

In [231]:
tsugar_points = {0: [-1, 4.5], 1: [4.5, 9], 2: [9, 13.5], 3: [13.5, 18], 
                 4: [18, 22.5], 5: [22.5, 27], 6: [27, 31], 7: [31, 36],
                 8: [36, 40], 9: [40, 45], 10: [45]}
food_noalcnovit.loc[:, 'tsugar_a_points'] = np.array(len(food_noalcnovit) * [0])
for points in tsugar_points:
    interval = tsugar_points[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['Totalsugarsg_per_100_g'] > interval[0]) & (food_noalcnovit['Totalsugarsg_per_100_g'] <= interval[1]),'tsugar_a_points'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['Totalsugarsg_per_100_g'] > interval[0]), 'tsugar_a_points'] = points

In [232]:
sodium_points = {0: [-1, 90], 1: [90, 180], 2: [180, 270], 3: [270, 360], 
                 4: [360, 450], 5: [450, 540], 6: [540, 630], 7: [630, 720],
                 8: [720, 810], 9: [810, 900], 10: [900]}
food_noalcnovit.loc[:, 'sodium_a_points'] = np.array(len(food_noalcnovit) * [0])
for points in sodium_points:
    interval = sodium_points[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['Sodiummg_per_100_g'] > interval[0]) & (food_noalcnovit['Sodiummg_per_100_g'] <= interval[1]),'sodium_a_points'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['Sodiummg_per_100_g'] > interval[0]), 'sodium_a_points'] = points

Calculate total A POINTS

In [233]:
food_noalcnovit.loc[:, 'total_a_points'] = food_noalcnovit["energy_a_points"] + food_noalcnovit["satfat_a_points"] + food_noalcnovit["tsugar_a_points"] + food_noalcnovit["sodium_a_points"] 

C POINTS

In [234]:
smoothies = {0: [-1, 40], 1: [40, 60], 2: [60, 80], 5: [80]}
food_noalcnovit.loc [:, 'smoothies'] = np.array(len(food_noalcnovit) * [0])
for points in smoothies:
    interval = smoothies[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['Smoothies_per_100_g'] > interval[0]) & (food_noalcnovit['Smoothies_per_100_g'] <= interval[1]),'smoothies'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['Smoothies_per_100_g'] > interval[0]), 'smoothies'] = points

In [235]:
fruit_juices = {0: [-1, 40], 1: [40, 60], 2: [60, 80], 5: [80]}
food_noalcnovit.loc [:, 'fruit_juice'] = np.array(len(food_noalcnovit) * [0])
for points in fruit_juices:
    interval = fruit_juices[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['Fruitjuice_per_100_g'] > interval[0]) & (food_noalcnovit['Fruitjuice_per_100_g'] <= interval[1]),'fruit_juice'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['Fruitjuice_per_100_g'] > interval[0]), 'fruit_juice'] = points

In [236]:
fvn_points = {0: [-1, 40], 1: [40, 60], 2: [60, 80], 5: [80]} 
food_noalcnovit.loc[:, 'fvn_c_points'] = np.array(len(food_noalcnovit) * [0])
for points in fvn_points:
    interval = fvn_points[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['fvn_per_100_g'] > interval[0]) & (food_noalcnovit['fvn_per_100_g'] <= interval[1]),'fvn_c_points'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['fvn_per_100_g'] > interval[0]), 'fvn_c_points'] = points

In [237]:
aoacfibre_points = {0: [-1, 0.9], 1: [0.9, 1.9], 2: [1.9, 2.8], 3: [2.8, 3.7], 4: [3.7, 4.7], 5: [4.7]} 

food_noalcnovit.loc[:, 'aoacfibre_c_points'] = np.array(len(food_noalcnovit) * [0])
for points in aoacfibre_points:
    interval = aoacfibre_points[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['AOACFibreg_per_100_g'] > interval[0]) & (food_noalcnovit['AOACFibreg_per_100_g'] <= interval[1]),'aoacfibre_c_points'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['AOACFibreg_per_100_g'] > interval[0]), 'aoacfibre_c_points'] = points

Impose some conditions on proteins here: If a food or drink scores 11 or more ‘A’ points then it cannot score points for protein unless it also scores 5 points for fruit, vegetables and nuts.

In [238]:
protein_points = {0: [-1, 1.6], 1: [1.6, 3.2], 2: [3.2, 4.8], 3: [4.8, 6.4], 4: [6.4, 8.0], 5: [8.0]} 

food_noalcnovit.loc[:, 'protein_c_points'] = np.array(len(food_noalcnovit) * [0])
for points in protein_points:
    interval = protein_points[points]
    if len(interval) == 2:
        food_noalcnovit.loc[(food_noalcnovit['Proteing_per_100_g'] > interval[0]) & (food_noalcnovit['Proteing_per_100_g'] <= interval[1])  & (food_noalcnovit['total_a_points'] < 11),'protein_c_points'] = points
        food_noalcnovit.loc[(food_noalcnovit['Proteing_per_100_g'] > interval[0]) & (food_noalcnovit['Proteing_per_100_g'] <= interval[1])  & (food_noalcnovit['total_a_points'] >= 11) & (food_noalcnovit['fvn_c_points'] >= 5),'protein_c_points'] = points
    elif len(interval) == 1:
        food_noalcnovit.loc[(food_noalcnovit['Proteing_per_100_g'] > interval[0])  & (food_noalcnovit['total_a_points'] < 11), 'protein_c_points'] = points
        food_noalcnovit.loc[(food_noalcnovit['Proteing_per_100_g'] > interval[0])  & (food_noalcnovit['total_a_points'] >= 11) & (food_noalcnovit['fvn_c_points'] >= 5), 'protein_c_points'] = points

Calculate total C POINTS

In [239]:
food_noalcnovit.loc[:, 'total_c_points'] = food_noalcnovit["fvn_c_points"] + food_noalcnovit["aoacfibre_c_points"] + food_noalcnovit["protein_c_points"] +food_noalcnovit['fruit_juice']+ food_noalcnovit['smoothies']

TOTAL POINTS

In [240]:
food_noalcnovit.loc[:, "total_points"] = food_noalcnovit["total_a_points"]-food_noalcnovit ["total_c_points"]

For food: less healthy (HFSS==1) is assigned when total points is 4 or greater

In [241]:
food_noalcnovit.loc[:, "lesshealthy"] = food_noalcnovit["total_points"] >= 4 

Drinks need to be done separately, and they are calculated below.

Create a drink set - include all drink names (Remove fruit juice and smoothies since they are calculated elsewhere)

In [242]:
drink_set={'SEMI SKIMMED MILK','SKIMMED MILK', 'SOFT DRINKS NOT LOW CALORIE', 'SOFT DRINKS LOW CALORIE', 'COMMERCIAL TODDLERS FOODS AND DRINKS', 
       '1% Fat Milk', 'TEA COFFEE AND WATER', 'WHOLE MILK'} 

We are also including drinking yoghurt into the drinks list, so creating another drinks set for this

In [243]:
drink_set2={'ACTIMEL PROBIOTIC DRINKING YOGURT',
'ACTIMEL PROBIOTIC YOGURT DRINK 0.1% FAT',
'BENECOL PLUS HEART YOGURT DRINK', 'BENECOL YOGURT DRINKS',
'BENECOL YOGURT DRINKS, CONTAINING STANOLS',
'CHILDRENS YOGURT DRINK WITH OMEGA 3, CALCIUM AND VITAMIN D',
'CHILDRENS YOGURT DRINK WITH VITAMIN D',
'MULLER VITALITY PROBIOTIC DRINK',
'MULLER VITALITY PROBIOTIC DRINK WITH OMEGA 3',
'OPTIFIT YOGURT DRINKS, ANY FLAVOUR, FORTIFIED WITH VITS C, E, B6',
'SMOOTHIES WITH FRUIT AND DAIRY PRODUCTS, BOTTLED, PURCHASED, NOT FORTIFIED',
'YAKULT', 'YOGURT DRINK', 'YOGURT DRINK CONTAINING FRUIT PUREE',
'YOGURT DRINK FORTIFIED WITH VITAMINS B6, C AND D'}

Drinks are less healthy (or HFSS) if the total points is 1 or greater

In [244]:
food_noalcnovit.loc[food_noalcnovit['MainFoodGroupDesc'].isin(drink_set), 'lesshealthy'] = food_noalcnovit.loc[food_noalcnovit['MainFoodGroupDesc'].isin(drink_set), "total_points"]>= 1 

In [245]:
food_noalcnovit.loc[food_noalcnovit['FoodName'].isin(drink_set2), 'lesshealthy'] = food_noalcnovit.loc[food_noalcnovit['FoodName'].isin(drink_set2), "total_points"]>= 1 

Calculate the proportion of foods and drinks that are less healthy (or HFSS)

In [246]:
np.sum(food_noalcnovit["lesshealthy"])/len(food_noalcnovit["lesshealthy"])

0.334097810720177

Calculate the proportion of foods and drinks that are UPF

In [247]:
np.sum(food_noalcnovit["NOVA_UPF_NUPF_boolean"])/len(food_noalcnovit["NOVA_UPF_NUPF_boolean"])

0.3621039234567016

Calculate the proportion of UPFs that are also HFSS

In [248]:
np.sum((food_noalcnovit["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit["lesshealthy"]))/np.sum (food_noalcnovit["NOVA_UPF_NUPF_boolean"])

0.5561558953993395

Here we quantify the proportion of food and drink items from NDNS that are: neither HFSS (NPM 2004/5) nor UPFs; UPFs only;
HFSS only (NPM 2004/5); HFSS (NPM 2004/5) and UPFs based on the overall sample.
For the summary graphs aggregate for food and drinks

In [249]:
value_notupf_healthy = np.sum((~food_noalcnovit["NOVA_UPF_NUPF_boolean"])*~food_noalcnovit["lesshealthy"])/len(food_noalcnovit)
value_upf_healthy = np.sum((food_noalcnovit["NOVA_UPF_NUPF_boolean"])*~food_noalcnovit["lesshealthy"])/len(food_noalcnovit)
value_notupf_lesshealthy = np.sum((~food_noalcnovit["NOVA_UPF_NUPF_boolean"])*food_noalcnovit["lesshealthy"])/len(food_noalcnovit)
value_upf_lesshealthy = np.sum((food_noalcnovit["NOVA_UPF_NUPF_boolean"])*food_noalcnovit["lesshealthy"])/len(food_noalcnovit)
print (value_notupf_healthy)
print (value_upf_healthy)
print (value_notupf_lesshealthy)
print (value_upf_lesshealthy)

0.5051844976007972
0.16071769167902586
0.1327115789425012
0.20138623177767578


Create a table with UPF sample only

In [250]:
food_noalcnovit_upf = food_noalcnovit[food_noalcnovit['NOVA_UPF_NUPF_boolean']]

We also want to check what would be the percentage increase in HFSS and UPF overlap if we include artificial sweeteners and soft drinks, low calorie. 

In [251]:
food_noalcnovit_upf.loc[:,"LH_assd"] = (food_noalcnovit_upf["lesshealthy"])| (food_noalcnovit_upf['MainFoodGroupDesc'] == "ARTIFICIAL SWEETENERS")|(food_noalcnovit_upf['MainFoodGroupDesc'] == "SOFT DRINKS LOW CALORIE")

The proportion of UPFs that would be captured if the policy added artificial sweeteners and soft drinks, low calorie

In [252]:
np.sum(food_noalcnovit_upf["LH_assd"] )/len(food_noalcnovit_upf.loc[:,"LH_assd"] )

0.6664485224989106

Proportion of total energy among UPF sample

In [253]:
np.sum(food_noalcnovit_upf["LH_assd"]* food_noalcnovit_upf['Energykcal'])/np.sum(food_noalcnovit_upf['Energykcal'])

0.5883347709564386

Proportion of total weight among UPF sample

In [254]:
np.sum(food_noalcnovit_upf["LH_assd"]* food_noalcnovit_upf['TotalGrams'])/np.sum(food_noalcnovit_upf['TotalGrams'])

0.6018477071914877

The proportion of energy from HFSS foods 

In [255]:
np.sum((food_noalcnovit["lesshealthy"]* food_noalcnovit['Energykcal']))/np.sum(food_noalcnovit['Energykcal'])

0.47368590501035823

The proportion of energy from HFSS foods among UPF foods

In [256]:
np.sum((food_noalcnovit_upf["lesshealthy"]* food_noalcnovit_upf['Energykcal']))/np.sum(food_noalcnovit_upf['Energykcal'])#len(food_noalcnovit[food_noalcnovit["NOVA_UPF_NUPF_boolean"]])

0.5868196013190269

The proportion of energy from UPF foods 

In [257]:
np.sum((food_noalcnovit["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit['Energykcal']))/np.sum(food_noalcnovit["Energykcal"])

0.5979799557987135

Proportion of energy derived from: neither HFSS (NPM 2004/5) nor UPFs; UPFs only; HFSS only (NPM 2004/5); HFSS (NPM 2004/5) and UPFs based on the overall sample.

In [258]:
value_notupf_healthy = np.sum(((~food_noalcnovit["NOVA_UPF_NUPF_boolean"]))*(~food_noalcnovit["lesshealthy"])*(food_noalcnovit['Energykcal']))/np.sum(food_noalcnovit['Energykcal'])
value_upf_healthy = np.sum(((food_noalcnovit["NOVA_UPF_NUPF_boolean"]))*(~food_noalcnovit["lesshealthy"])*(food_noalcnovit['Energykcal']))/np.sum(food_noalcnovit['Energykcal'])
value_notupf_lesshealthy = np.sum(((~food_noalcnovit["NOVA_UPF_NUPF_boolean"]))*((food_noalcnovit["lesshealthy"]))*(food_noalcnovit['Energykcal']))/np.sum(food_noalcnovit['Energykcal'])
value_upf_lesshealthy = np.sum(((food_noalcnovit["NOVA_UPF_NUPF_boolean"]))*(food_noalcnovit["lesshealthy"])*(food_noalcnovit['Energykcal']))/np.sum(food_noalcnovit['Energykcal'])
print (value_notupf_healthy)
print (value_upf_healthy)
print (value_notupf_lesshealthy)
print (value_upf_lesshealthy)

0.2792404984494982
0.24707359654014313
0.12277954575178796
0.35090635925857033


Proportion of weight derived from HFSS among UPF sample

In [259]:
np.sum((food_noalcnovit_upf["lesshealthy"]* food_noalcnovit_upf['TotalGrams']))/np.sum(food_noalcnovit_upf['TotalGrams'])#len(food_noalcnovit[food_noalcnovit["NOVA_UPF_NUPF_boolean"]])

0.383452868573104

Proportion of weight derived from HFSS in all sample

In [260]:
np.sum((food_noalcnovit["lesshealthy"]* food_noalcnovit['TotalGrams']))/np.sum(food_noalcnovit['TotalGrams'])

0.15999906050738835

Proportion of weight derived from UPFs in all sample

In [261]:
np.sum((food_noalcnovit["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit['TotalGrams']))/np.sum(food_noalcnovit["TotalGrams"])

0.32859279799964913

Proportion of weight derived from: neither HFSS (NPM 2004/5) nor UPFs; UPFs only; HFSS only (NPM 2004/5); HFSS (NPM 2004/5) and UPFs based on the overall sample.

In [262]:
value_notupf_healthy = np.sum(((~food_noalcnovit["NOVA_UPF_NUPF_boolean"]))*(~food_noalcnovit["lesshealthy"])*(food_noalcnovit['TotalGrams']))/np.sum(food_noalcnovit['TotalGrams'])
value_upf_healthy = np.sum(((food_noalcnovit["NOVA_UPF_NUPF_boolean"]))*(~food_noalcnovit["lesshealthy"])*(food_noalcnovit['TotalGrams']))/np.sum(food_noalcnovit['TotalGrams'])
value_notupf_lesshealthy = np.sum(((~food_noalcnovit["NOVA_UPF_NUPF_boolean"]))*((food_noalcnovit["lesshealthy"]))*(food_noalcnovit['TotalGrams']))/np.sum(food_noalcnovit['TotalGrams'])
value_upf_lesshealthy = np.sum(((food_noalcnovit["NOVA_UPF_NUPF_boolean"]))*(food_noalcnovit["lesshealthy"])*(food_noalcnovit['TotalGrams']))/np.sum(food_noalcnovit['TotalGrams'])
print (value_notupf_healthy)
print (value_upf_healthy)
print (value_notupf_lesshealthy)
print (value_upf_lesshealthy)

0.6374079924783908
0.20259294701422118
0.03399920952196041
0.12599985098542787


Examples of HFSS foods only (i.e., not UPFs), summaries with 'MainFoodGroupDesc' variable

In [263]:
food_groups_values = {}
for x in food_noalcnovit['MainFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["MainFoodGroupDesc"] == x)*(food_noalcnovit["lesshealthy"])*(~food_noalcnovit["NOVA_UPF_NUPF_boolean"]))/np.sum(food_noalcnovit["MainFoodGroupDesc"] == x)
    food_groups_values[x] = value   
    
food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]
    print (f'x: {x}, value: {value:.5f}')

x: BROWN GRANARY AND WHEATGERM BREAD, value: 0.00000
x: OTHER BREAD, value: 0.00000
x: WHITE BREAD, value: 0.00000
x: WHOLEMEAL BREAD, value: 0.00000
x: OTHER BREAKFAST CEREALS, value: 0.00000
x: CRISPS AND SAVOURY SNACKS, value: 0.00000
x: ICE CREAM, value: 0.00000
x: SEMI SKIMMED MILK, value: 0.00000
x: SKIMMED MILK, value: 0.00000
x: COATED CHICKEN, value: 0.00000
x: MEAT PIES AND PASTRIES, value: 0.00000
x: LIVER & DISHES, value: 0.00000
x: BURGERS AND KEBABS, value: 0.00000
x: FRUIT JUICE, value: 0.00000
x: ARTIFICIAL SWEETENERS, value: 0.00000
x: SUGAR CONFECTIONERY, value: 0.00000
x: CHOCOLATE CONFECTIONERY, value: 0.00000
x: SOFT DRINKS LOW CALORIE, value: 0.00000
x: REDUCED FAT SPREAD, value: 0.00000
x: SMOOTHIES 100% FRUIT AND/OR JUICE, value: 0.00000
x: 1% Fat Milk, value: 0.00000
x: CHIPS FRIED & ROAST POTATOES AND POTATO PRODUCTS, value: 0.00004
x: SOFT DRINKS NOT LOW CALORIE, value: 0.00042
x: TEA COFFEE AND WATER, value: 0.00055
x: SALAD AND OTHER RAW VEGETABLES, value: 

Examples of both UPF and HFSS based on all foods, summaries with 'MainFoodGroupDesc' variable

In [264]:
food_groups_values = {}
for x in food_noalcnovit['MainFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["MainFoodGroupDesc"] == x)*(food_noalcnovit["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit["lesshealthy"]))/np.sum(food_noalcnovit["MainFoodGroupDesc"] == x)
    food_groups_values[x] = value   
    
food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]
    print (f'x: {x}, value: {value:.5f}')

x: WHOLE MILK, value: 0.00000
x: SEMI SKIMMED MILK, value: 0.00000
x: SKIMMED MILK, value: 0.00000
x: SMOOTHIES 100% FRUIT AND/OR JUICE, value: 0.00000
x: 1% Fat Milk, value: 0.00000
x: BROWN GRANARY AND WHEATGERM BREAD, value: 0.00039
x: BUTTER, value: 0.00084
x: PUFA MARGARINE & OILS, value: 0.00091
x: TEA COFFEE AND WATER, value: 0.00164
x: FRUIT, value: 0.00196
x: VEGETABLES NOT RAW, value: 0.00247
x: WHITE FISH COATED OR FRIED, value: 0.00526
x: BEEF VEAL AND DISHES, value: 0.00743
x: OTHER POTATOES POTATO SALADS & DISHES, value: 0.00975
x: FRUIT JUICE, value: 0.00998
x: OTHER WHITE FISH SHELLFISH & FISH DISHES, value: 0.01127
x: LAMB AND DISHES, value: 0.01132
x: SALAD AND OTHER RAW VEGETABLES, value: 0.01216
x: WHOLEMEAL BREAD, value: 0.01912
x: OTHER MARGARINE FATS AND OILS, value: 0.02316
x: CHIPS FRIED & ROAST POTATOES AND POTATO PRODUCTS, value: 0.03503
x: NUTS AND SEEDS, value: 0.04509
x: EGGS AND EGG DISHES, value: 0.06593
x: WHITE BREAD, value: 0.08015
x: OILY FISH, value

Examples of both UPF and not HFSS based on all foods, summaries with 'MainFoodGroupDesc' variable

In [265]:
food_groups_values = {}
for x in food_noalcnovit['MainFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["MainFoodGroupDesc"] == x)*(food_noalcnovit["NOVA_UPF_NUPF_boolean"])*(~food_noalcnovit["lesshealthy"]))/np.sum(food_noalcnovit["MainFoodGroupDesc"] == x)
    food_groups_values[x] = value   
    
food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]
    print (f'x: {x}, value: {value:.5f}')

x: WHOLE MILK, value: 0.00000
x: SEMI SKIMMED MILK, value: 0.00000
x: BUTTER, value: 0.00000
x: OTHER MARGARINE FATS AND OILS, value: 0.00000
x: NUTS AND SEEDS, value: 0.00000
x: CHOCOLATE CONFECTIONERY, value: 0.00000
x: LOW FAT SPREAD, value: 0.00000
x: REDUCED FAT SPREAD, value: 0.00000
x: 1% Fat Milk, value: 0.00000
x: FRUIT, value: 0.00009
x: SUGARS PRESERVES AND SWEET SPREADS, value: 0.00018
x: PUFA MARGARINE & OILS, value: 0.00045
x: ICE CREAM, value: 0.00361
x: SKIMMED MILK, value: 0.00599
x: BACON AND HAM, value: 0.00613
x: CHEESE, value: 0.00753
x: LIVER & DISHES, value: 0.00764
x: OTHER MEAT AND MEAT PRODUCTS, value: 0.01048
x: SALAD AND OTHER RAW VEGETABLES, value: 0.01326
x: TEA COFFEE AND WATER, value: 0.01417
x: EGGS AND EGG DISHES, value: 0.02286
x: SAUSAGES, value: 0.02414
x: CRISPS AND SAVOURY SNACKS, value: 0.02672
x: FRUIT JUICE, value: 0.03320
x: PORK AND DISHES, value: 0.03985
x: LAMB AND DISHES, value: 0.05874
x: BUNS CAKES PASTRIES & FRUIT PIES, value: 0.06221
x

Examples of UPFs based on all foods, summaries with 'MainFoodGroupDesc' variable

In [266]:
food_groups_values = {}
for x in food_noalcnovit['MainFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["MainFoodGroupDesc"] == x)*(food_noalcnovit["NOVA_UPF_NUPF_boolean"]))/np.sum(food_noalcnovit["MainFoodGroupDesc"] == x)
    food_groups_values[x] = value   
    
food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]
    print (f'x: {x}, value: {value:.5f}')

x: WHOLE MILK, value: 0.00000
x: SEMI SKIMMED MILK, value: 0.00000
x: 1% Fat Milk, value: 0.00000
x: BUTTER, value: 0.00084
x: PUFA MARGARINE & OILS, value: 0.00136
x: FRUIT, value: 0.00205
x: SKIMMED MILK, value: 0.00599
x: TEA COFFEE AND WATER, value: 0.01581
x: OTHER MARGARINE FATS AND OILS, value: 0.02316
x: SALAD AND OTHER RAW VEGETABLES, value: 0.02542
x: FRUIT JUICE, value: 0.04317
x: NUTS AND SEEDS, value: 0.04509
x: LAMB AND DISHES, value: 0.07006
x: OTHER WHITE FISH SHELLFISH & FISH DISHES, value: 0.08313
x: EGGS AND EGG DISHES, value: 0.08879
x: OTHER POTATOES POTATO SALADS & DISHES, value: 0.08895
x: VEGETABLES NOT RAW, value: 0.09018
x: BEEF VEAL AND DISHES, value: 0.13289
x: CHICKEN AND TURKEY DISHES, value: 0.15752
x: SUGARS PRESERVES AND SWEET SPREADS, value: 0.16458
x: CHEESE, value: 0.16712
x: PORK AND DISHES, value: 0.17365
x: WHITE FISH COATED OR FRIED, value: 0.22116
x: PASTA RICE AND OTHER CEREALS, value: 0.24941
x: OILY FISH, value: 0.28822
x: MISCELLANEOUS, valu

Examples of neither UPF nor HFSS based on all foods, summaries with 'MainFoodGroupDesc' variable

In [267]:
food_groups_values = {}
for x in food_noalcnovit['MainFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["MainFoodGroupDesc"] == x)*(~food_noalcnovit["NOVA_UPF_NUPF_boolean"])*(~food_noalcnovit["lesshealthy"]))/np.sum(food_noalcnovit["MainFoodGroupDesc"] == x)
    food_groups_values[x] = value   
    
food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]
    print (f'x: {x}, value: {value:.5f}')

x: BROWN GRANARY AND WHEATGERM BREAD, value: 0.00000
x: OTHER BREAD, value: 0.00000
x: WHITE BREAD, value: 0.00000
x: WHOLEMEAL BREAD, value: 0.00000
x: OTHER BREAKFAST CEREALS, value: 0.00000
x: BISCUITS, value: 0.00000
x: CRISPS AND SAVOURY SNACKS, value: 0.00000
x: ICE CREAM, value: 0.00000
x: BUTTER, value: 0.00000
x: OTHER MARGARINE FATS AND OILS, value: 0.00000
x: PUFA MARGARINE & OILS, value: 0.00000
x: BACON AND HAM, value: 0.00000
x: COATED CHICKEN, value: 0.00000
x: MEAT PIES AND PASTRIES, value: 0.00000
x: SAUSAGES, value: 0.00000
x: BURGERS AND KEBABS, value: 0.00000
x: ARTIFICIAL SWEETENERS, value: 0.00000
x: SUGAR CONFECTIONERY, value: 0.00000
x: CHOCOLATE CONFECTIONERY, value: 0.00000
x: SOFT DRINKS LOW CALORIE, value: 0.00000
x: LOW FAT SPREAD, value: 0.00000
x: REDUCED FAT SPREAD, value: 0.00000
x: SUGARS PRESERVES AND SWEET SPREADS, value: 0.00016
x: SOFT DRINKS NOT LOW CALORIE, value: 0.00042
x: BUNS CAKES PASTRIES & FRUIT PIES, value: 0.00178
x: COMMERCIAL TODDLERS 

BY ENERGY

Examples of UPFs weighted by energy, summaries with 'MainFoodGroupDesc' variable

In [268]:
food_groups_values = {}
for x in food_noalcnovit['MainFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["MainFoodGroupDesc"] == x)*(food_noalcnovit["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit['Energykcal']))/np.sum (food_noalcnovit["Energykcal"])
    food_groups_values[x] = value  

food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]
    print (f'x: {x}, value: {value:.5f}')

x: WHOLE MILK, value: 0.00000
x: SEMI SKIMMED MILK, value: 0.00000
x: 1% Fat Milk, value: 0.00000
x: PUFA MARGARINE & OILS, value: 0.00001
x: SKIMMED MILK, value: 0.00001
x: BUTTER, value: 0.00002
x: ARTIFICIAL SWEETENERS, value: 0.00004
x: FRUIT, value: 0.00013
x: OTHER MARGARINE FATS AND OILS, value: 0.00036
x: LAMB AND DISHES, value: 0.00053
x: NUTS AND SEEDS, value: 0.00056
x: LIVER & DISHES, value: 0.00062
x: COMMERCIAL TODDLERS FOODS AND DRINKS, value: 0.00066
x: FRUIT JUICE, value: 0.00073
x: SMOOTHIES 100% FRUIT AND/OR JUICE, value: 0.00076
x: TEA COFFEE AND WATER, value: 0.00085
x: OTHER WHITE FISH SHELLFISH & FISH DISHES, value: 0.00114
x: SOFT DRINKS LOW CALORIE, value: 0.00118
x: PORK AND DISHES, value: 0.00124
x: SALAD AND OTHER RAW VEGETABLES, value: 0.00153
x: OILY FISH, value: 0.00196
x: OTHER BREAD, value: 0.00235
x: OTHER POTATOES POTATO SALADS & DISHES, value: 0.00240
x: LOW FAT SPREAD, value: 0.00253
x: WHITE FISH COATED OR FRIED, value: 0.00260
x: CHEESE, value: 0.

Examples of HFSS ONLY weighted by energy, summaries with 'MainFoodGroupDesc' variable

In [269]:
food_groups_values = {}
for x in food_noalcnovit['MainFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["MainFoodGroupDesc"] == x)*(food_noalcnovit["lesshealthy"])*(~food_noalcnovit["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit['Energykcal']))/np.sum (food_noalcnovit["Energykcal"])
    food_groups_values[x] = value 
    
food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]    
    print (f'x: {x}, value: {value:.5f}')

x: BROWN GRANARY AND WHEATGERM BREAD, value: 0.00000
x: OTHER BREAD, value: 0.00000
x: WHITE BREAD, value: 0.00000
x: WHOLEMEAL BREAD, value: 0.00000
x: OTHER BREAKFAST CEREALS, value: 0.00000
x: CRISPS AND SAVOURY SNACKS, value: 0.00000
x: ICE CREAM, value: 0.00000
x: SEMI SKIMMED MILK, value: 0.00000
x: SKIMMED MILK, value: 0.00000
x: COATED CHICKEN, value: 0.00000
x: MEAT PIES AND PASTRIES, value: 0.00000
x: LIVER & DISHES, value: 0.00000
x: BURGERS AND KEBABS, value: 0.00000
x: FRUIT JUICE, value: 0.00000
x: ARTIFICIAL SWEETENERS, value: 0.00000
x: SUGAR CONFECTIONERY, value: 0.00000
x: CHOCOLATE CONFECTIONERY, value: 0.00000
x: SOFT DRINKS LOW CALORIE, value: 0.00000
x: REDUCED FAT SPREAD, value: 0.00000
x: SMOOTHIES 100% FRUIT AND/OR JUICE, value: 0.00000
x: 1% Fat Milk, value: 0.00000
x: CHIPS FRIED & ROAST POTATOES AND POTATO PRODUCTS, value: 0.00000
x: SOFT DRINKS NOT LOW CALORIE, value: 0.00000
x: COMMERCIAL TODDLERS FOODS AND DRINKS, value: 0.00000
x: LOW FAT SPREAD, value: 

Proportion of weight derived from UPFs ONLY in each food category, summaries with 'MainFoodGroupDesc' variable

In [270]:
food_groups_values = {}
for x in food_noalcnovit['MainFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["MainFoodGroupDesc"] == x)*(food_noalcnovit["NOVA_UPF_NUPF_boolean"])*(~food_noalcnovit["lesshealthy"])*(food_noalcnovit['Energykcal']))/np.sum (food_noalcnovit["Energykcal"])
    food_groups_values[x] = value 
    
food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]    
    print (f'x: {x}, value: {value:.5f}')

x: WHOLE MILK, value: 0.00000
x: SEMI SKIMMED MILK, value: 0.00000
x: BUTTER, value: 0.00000
x: OTHER MARGARINE FATS AND OILS, value: 0.00000
x: NUTS AND SEEDS, value: 0.00000
x: CHOCOLATE CONFECTIONERY, value: 0.00000
x: LOW FAT SPREAD, value: 0.00000
x: REDUCED FAT SPREAD, value: 0.00000
x: 1% Fat Milk, value: 0.00000
x: SUGARS PRESERVES AND SWEET SPREADS, value: 0.00000
x: FRUIT, value: 0.00000
x: PUFA MARGARINE & OILS, value: 0.00000
x: LIVER & DISHES, value: 0.00001
x: ARTIFICIAL SWEETENERS, value: 0.00001
x: SKIMMED MILK, value: 0.00001
x: ICE CREAM, value: 0.00002
x: SUGAR CONFECTIONERY, value: 0.00003
x: BACON AND HAM, value: 0.00004
x: CHEESE, value: 0.00005
x: OTHER MEAT AND MEAT PRODUCTS, value: 0.00011
x: COMMERCIAL TODDLERS FOODS AND DRINKS, value: 0.00025
x: PORK AND DISHES, value: 0.00035
x: SAUSAGES, value: 0.00041
x: LAMB AND DISHES, value: 0.00045
x: EGGS AND EGG DISHES, value: 0.00050
x: FRUIT JUICE, value: 0.00065
x: TEA COFFEE AND WATER, value: 0.00067
x: CRISPS AN

Overlap among food categories between UPF and HFSS weighted by energy

In [271]:
food_groups_values = {}
for x in food_noalcnovit['MainFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["MainFoodGroupDesc"] == x)*(food_noalcnovit["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit["lesshealthy"])*(food_noalcnovit['Energykcal']))/np.sum (food_noalcnovit["Energykcal"])
    food_groups_values[x] = value 
    
food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]    
    print (f'x: {x}, value: {value:.5f}')

x: WHOLE MILK, value: 0.00000
x: SEMI SKIMMED MILK, value: 0.00000
x: SKIMMED MILK, value: 0.00000
x: SMOOTHIES 100% FRUIT AND/OR JUICE, value: 0.00000
x: 1% Fat Milk, value: 0.00000
x: BROWN GRANARY AND WHEATGERM BREAD, value: 0.00000
x: PUFA MARGARINE & OILS, value: 0.00001
x: BUTTER, value: 0.00002
x: ARTIFICIAL SWEETENERS, value: 0.00003
x: WHITE FISH COATED OR FRIED, value: 0.00007
x: BEEF VEAL AND DISHES, value: 0.00007
x: OTHER WHITE FISH SHELLFISH & FISH DISHES, value: 0.00008
x: FRUIT JUICE, value: 0.00008
x: LAMB AND DISHES, value: 0.00009
x: FRUIT, value: 0.00012
x: TEA COFFEE AND WATER, value: 0.00018
x: SOFT DRINKS LOW CALORIE, value: 0.00028
x: OTHER MARGARINE FATS AND OILS, value: 0.00036
x: OTHER POTATOES POTATO SALADS & DISHES, value: 0.00038
x: OTHER BREAD, value: 0.00039
x: COMMERCIAL TODDLERS FOODS AND DRINKS, value: 0.00041
x: NUTS AND SEEDS, value: 0.00056
x: VEGETABLES NOT RAW, value: 0.00059
x: OILY FISH, value: 0.00060
x: LIVER & DISHES, value: 0.00061
x: WHOLE

BY WEIGHT

Proportion of weight derived from HFSS ONLY in each food category

In [272]:
food_groups_values = {}
for x in food_noalcnovit['MainFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["MainFoodGroupDesc"] == x)*(food_noalcnovit["lesshealthy"])*(~food_noalcnovit["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit['TotalGrams']))/np.sum (food_noalcnovit["TotalGrams"])
    food_groups_values[x] = value 
    
food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]    
    print (f'x: {x}, value: {value:.5f}')

x: BROWN GRANARY AND WHEATGERM BREAD, value: 0.00000
x: OTHER BREAD, value: 0.00000
x: WHITE BREAD, value: 0.00000
x: WHOLEMEAL BREAD, value: 0.00000
x: OTHER BREAKFAST CEREALS, value: 0.00000
x: CRISPS AND SAVOURY SNACKS, value: 0.00000
x: ICE CREAM, value: 0.00000
x: SEMI SKIMMED MILK, value: 0.00000
x: SKIMMED MILK, value: 0.00000
x: COATED CHICKEN, value: 0.00000
x: MEAT PIES AND PASTRIES, value: 0.00000
x: LIVER & DISHES, value: 0.00000
x: BURGERS AND KEBABS, value: 0.00000
x: FRUIT JUICE, value: 0.00000
x: ARTIFICIAL SWEETENERS, value: 0.00000
x: SUGAR CONFECTIONERY, value: 0.00000
x: CHOCOLATE CONFECTIONERY, value: 0.00000
x: SOFT DRINKS LOW CALORIE, value: 0.00000
x: REDUCED FAT SPREAD, value: 0.00000
x: SMOOTHIES 100% FRUIT AND/OR JUICE, value: 0.00000
x: 1% Fat Milk, value: 0.00000
x: CHIPS FRIED & ROAST POTATOES AND POTATO PRODUCTS, value: 0.00000
x: COMMERCIAL TODDLERS FOODS AND DRINKS, value: 0.00000
x: LOW FAT SPREAD, value: 0.00000
x: SOFT DRINKS NOT LOW CALORIE, value: 

Proportion of weight derived from UPFs ONLY in each food category

In [273]:
food_groups_values = {}
for x in food_noalcnovit['MainFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["MainFoodGroupDesc"] == x)*(food_noalcnovit["NOVA_UPF_NUPF_boolean"])*(~food_noalcnovit["lesshealthy"])*(food_noalcnovit['TotalGrams']))/np.sum (food_noalcnovit["TotalGrams"])
    food_groups_values[x] = value 
    
food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]    
    print (f'x: {x}, value: {value:.5f}')

x: WHOLE MILK, value: 0.00000
x: SEMI SKIMMED MILK, value: 0.00000
x: BUTTER, value: 0.00000
x: OTHER MARGARINE FATS AND OILS, value: 0.00000
x: NUTS AND SEEDS, value: 0.00000
x: CHOCOLATE CONFECTIONERY, value: 0.00000
x: LOW FAT SPREAD, value: 0.00000
x: REDUCED FAT SPREAD, value: 0.00000
x: 1% Fat Milk, value: 0.00000
x: PUFA MARGARINE & OILS, value: 0.00000
x: SUGARS PRESERVES AND SWEET SPREADS, value: 0.00000
x: FRUIT, value: 0.00000
x: LIVER & DISHES, value: 0.00001
x: ICE CREAM, value: 0.00001
x: SKIMMED MILK, value: 0.00003
x: SUGAR CONFECTIONERY, value: 0.00003
x: BACON AND HAM, value: 0.00003
x: CHEESE, value: 0.00004
x: OTHER MEAT AND MEAT PRODUCTS, value: 0.00005
x: ARTIFICIAL SWEETENERS, value: 0.00011
x: CRISPS AND SAVOURY SNACKS, value: 0.00012
x: EGGS AND EGG DISHES, value: 0.00018
x: SAUSAGES, value: 0.00018
x: PORK AND DISHES, value: 0.00022
x: BISCUITS, value: 0.00026
x: LAMB AND DISHES, value: 0.00031
x: COMMERCIAL TODDLERS FOODS AND DRINKS, value: 0.00033
x: MEAT PI

Proportion of weight derived from UPF AND HFSS in each food category 

In [274]:
food_groups_values = {}
for x in food_noalcnovit['MainFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["MainFoodGroupDesc"] == x)*(food_noalcnovit["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit["lesshealthy"])*(food_noalcnovit['TotalGrams']))/np.sum (food_noalcnovit["TotalGrams"])
    food_groups_values[x] = value 
    
food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]    
    print (f'x: {x}, value: {value:.5f}')

x: WHOLE MILK, value: 0.00000
x: SEMI SKIMMED MILK, value: 0.00000
x: SKIMMED MILK, value: 0.00000
x: SMOOTHIES 100% FRUIT AND/OR JUICE, value: 0.00000
x: 1% Fat Milk, value: 0.00000
x: PUFA MARGARINE & OILS, value: 0.00000
x: BROWN GRANARY AND WHEATGERM BREAD, value: 0.00000
x: BUTTER, value: 0.00000
x: ARTIFICIAL SWEETENERS, value: 0.00001
x: LAMB AND DISHES, value: 0.00002
x: WHITE FISH COATED OR FRIED, value: 0.00002
x: FRUIT JUICE, value: 0.00003
x: OTHER MARGARINE FATS AND OILS, value: 0.00004
x: BEEF VEAL AND DISHES, value: 0.00004
x: OTHER WHITE FISH SHELLFISH & FISH DISHES, value: 0.00005
x: FRUIT, value: 0.00006
x: COMMERCIAL TODDLERS FOODS AND DRINKS, value: 0.00010
x: OTHER BREAD, value: 0.00011
x: OTHER POTATOES POTATO SALADS & DISHES, value: 0.00014
x: WHOLEMEAL BREAD, value: 0.00015
x: LIVER & DISHES, value: 0.00016
x: OILY FISH, value: 0.00016
x: VEGETABLES NOT RAW, value: 0.00017
x: NUTS AND SEEDS, value: 0.00020
x: TEA COFFEE AND WATER, value: 0.00026
x: PORK AND DISH

Create milk alternatives based on FoodNames

In [275]:
milk_alt_set2={'ALMOND ALTERNATIVE TO MILK FORTIFIED', 'ALMOND ALTERNATIVE TO MILK UNSWEETENED FORTIFIED',
              'ALMOND ALTERNATIVE TO MILK UNSWEETENED FORTIFIED WITH CALCIUM AND VITAMINS', 'ALMOND ALTERNATIVE TO MILK WITH CHOCOLATE', 
              'ALPRO RICE ALTERNATIVE TO MILK ORIGINAL', 'COCONUT ALTERNATIVE TO MILK SWEETENED CALCIUM ENRICHED', 
              'COFFEE CREAMER, LIQUID (GLUCOSE SYRUP AND VEGETABLE FAT)', 'COFFEE MATE POWDER' , 'COFFEE WHITENER , LIQUID (SKIMMED MILK AND NON MILK FAT)', 
              'COFFEE WHITENER POWDER LOW FAT DRY WEIGHT', 'COFFEE WHITENER POWDER, FULL FAT, DRY WEIGHT', 'EVAPORATED MILK LOW FAT CANNED', 
              'FLAVOURED MILK DRINKS, NAS, MADE WITH SEMI-SKIMMED MILK', 'HEMP ALTERNATIVE TO MILK', 'HOT CHOCOLATE (NO CREAM) SKIMMED MILK TAKEAWAY ONLY', 
              'HOT CHOCOLATE (NO CREAM) WHOLE MILK TAKEAWAY ONLY', 'HOT CHOCOLATE (WITH CREAM) WHOLE MILK TAKEAWAY ONLY', 
              'HOT CHOCOLATE MADE WITH SEMI-SKIMMED MILK', 'HOT CHOCOLATE MADE WITH SEMI-SKIMMED MILK AND EXTRA CREAM', 'LACTOFREE FRESH SKIMMED MILK FORTIFIED', 
              'LACTOSE FREE SEMI SKIMMED MILK', 'LACTOSE FREE WHOLE MILK', 'MARS BAR MILK', 'MILK CONDENSED SKIMMED SWEETENED', 'MILK CONDENSED WHOLE SWEETENED', 
              'MILK DRINK PASTEURISED/STERILISED CHOCOLATE FLAVOUR', 'MILK DRINK PASTEURISED/STERILISED NOT CHOCOLATE FLAVOUR', 'MILK EVAPORATED', 
              'MILK SHAKE THICK STYLE TAKEAWAY', 'MILK SHAKE WHOLE MILK WITH ICECREAM', 'MILKSHAKE', 'MILKSHAKE PURCHASED MADE WITH SEMI-SKIMMED MILK', 
              'MILKSHAKE PURCHASED MADE WITH SEMI-SKIMMED MILK, FORTIFIED', 'MILKSHAKE UHT PURCHASED MADE WITH WHOLEMILK', 
              'MILKSHAKE WITH SKIMMED MILK + ARTIFICIAL SWEETENERS', 'OAT BASED MILK ALTERNATIVE FORTIFIED', 'OAT BASED MILK ALTERNATIVE UNFORTIFIED', 
              'PROZERO PROTEIN FREE MILK', 'RICE DREAM ALTERNATIVE TO MILK, WITH ADDED CALCIUM', 'RICE DRINK ALTERNATIVE TO MILK (NOT FORTIFIED)', 
              'SAINSBURYS THICK MILK SHAKE PASTEURISED', 'SO GOOD, FORTIFIED SOYA DRINK', 'SOYA ALTERNATIVE TO MILK FLAVOURED', 
              'SOYA ALTERNATIVE TO MILK FLAVOURED FORTIFIED', 'SOYA ALTERNATIVE TO MILK SWEETENED CALCIUM ENRICHED', 
              'SOYA ALTERNATIVE TO MILK SWEETENED CALCIUM, IRON AND VITAMIN ENRICHED', 'SOYA ALTERNATIVE TO MILK SWEETENED PLAIN', 
              'SOYA ALTERNATIVE TO MILK UNSWEETENED','SOYA ALTERNATIVE TO MILK, FORTIFIED', 'SOYA ALTERNATIVE TO MILK, LIGHT, UNSWEETENED, FORTIFIED', 
              'SOYA CUSTARD WITH ADDED CALCIUM' , 'WEETABIX ON THE GO BREAKFAST DRINK FORTIFIED'} 

In [276]:
food_noalcnovit.loc[:, "LH_Milk_A2"] = (food_noalcnovit['FoodName'].isin(milk_alt_set2))

In [277]:
(food_noalcnovit["LH_Milk_A2"]).value_counts()

False    1722638
True        7520
Name: LH_Milk_A2, dtype: int64

Artificial sweeteners

In [278]:
food_noalcnovit.loc[:, "LH_AS"] = (food_noalcnovit['MainFoodGroupDesc'] == "ARTIFICIAL SWEETENERS")

HFSS and including artificial sweeteners and soft drinks low calorie 

In [279]:
food_noalcnovit_upf.loc[:,"LH_assd"] = (food_noalcnovit_upf["lesshealthy"])| (food_noalcnovit_upf['MainFoodGroupDesc'] == "ARTIFICIAL SWEETENERS")|(food_noalcnovit_upf['MainFoodGroupDesc'] == "SOFT DRINKS LOW CALORIE")

In [280]:
np.sum(food_noalcnovit_upf["LH_assd"] )/len(food_noalcnovit_upf.loc[:,"LH_assd"] )

0.6664485224989106

Proportion of UPFs based on more granular categories (i.e., SubFoodGroupDesc)

In [281]:
food_groups_values = {}
for x in food_noalcnovit['SubFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["SubFoodGroupDesc"] == x)*(food_noalcnovit["NOVA_UPF_NUPF_boolean"]))/np.sum(food_noalcnovit["SubFoodGroupDesc"] == x)
    food_groups_values[x] = value   
    
food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]
    print (f'x: {x}, value: {value:.5f}')

x: CEREAL BASED MILK PUDDINGS - HOMEMADE, value: 0.00000
x: WHOLE MILK, value: 0.00000
x: SEMI SKIMMED MILK, value: 0.00000
x: CHEDDAR CHEESE, value: 0.00000
x: COTTAGE CHEESE, value: 0.00000
x: OTHER COOKING FATS AND OILS NOT PUFA, value: 0.00000
x: OTHER OILY FISH INCLUDING HOMEMADE DISHES, value: 0.00000
x: CANNED FRUIT IN JUICE, value: 0.00000
x: LEAFY GREEN VEGETABLES NOT RAW, value: 0.00000
x: CARROTS NOT RAW, value: 0.00000
x: PEAS NOT RAW, value: 0.00000
x: TOMATOES RAW, value: 0.00000
x: APPLES AND PEARS NOT CANNED, value: 0.00000
x: CANNED FRUIT IN SYRUP, value: 0.00000
x: BANANAS, value: 0.00000
x: CITRUS FRUIT NOT CANNED, value: 0.00000
x: TAP WATER ONLY, value: 0.00000
x: HERBAL TEA (MADE-UP WEIGHT), value: 0.00000
x: 1% Fat Milk, value: 0.00000
x: TOMATOES NOT RAW, value: 0.00020
x: POLYUNSATURATED OILS, value: 0.00045
x: GREEN BEANS NOT RAW, value: 0.00065
x: BUTTER, value: 0.00084
x: OTHER WHITE FISH INCLUDING HOMEMADE DISHES, value: 0.00232
x: CARROTS RAW, value: 0.002

Proportion of HFSS based on more granular categories (i.e., SubFoodGroupDesc)

In [282]:
food_groups_values = {}
for x in food_noalcnovit['SubFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["SubFoodGroupDesc"] == x)*(food_noalcnovit["lesshealthy"]))/np.sum(food_noalcnovit["SubFoodGroupDesc"] == x)
    food_groups_values[x] = value   
    
food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]
    print (f'x: {x}, value: {value:.5f}')

x: SEMI SKIMMED MILK, value: 0.00000
x: SKIMMED MILK, value: 0.00000
x: COTTAGE CHEESE, value: 0.00000
x: MANUFACTURED LAMB PRODUCTS INCLUDING READY MEALS, value: 0.00000
x: BAKED BEANS, value: 0.00000
x: CANNED FRUIT IN JUICE, value: 0.00000
x: GREEN BEANS NOT RAW, value: 0.00000
x: LEAFY GREEN VEGETABLES NOT RAW, value: 0.00000
x: CARROTS RAW, value: 0.00000
x: CARROTS NOT RAW, value: 0.00000
x: PEAS NOT RAW, value: 0.00000
x: TOMATOES RAW, value: 0.00000
x: CANNED FRUIT IN SYRUP, value: 0.00000
x: CITRUS FRUIT NOT CANNED, value: 0.00000
x: BOTTLED WATER STILL OR CARBONATED, value: 0.00000
x: SOFT DRINKS LOW CALORIE CARBONATED, value: 0.00000
x: TAP WATER ONLY, value: 0.00000
x: HERBAL TEA (MADE-UP WEIGHT), value: 0.00000
x: SMOOTHIES 100% FRUIT AND/OR JUICE, value: 0.00000
x: 1% Fat Milk, value: 0.00000
x: READY MEALS BASED ON SAUSAGES, value: 0.00000
x: APPLES AND PEARS NOT CANNED, value: 0.00005
x: OTHER FRIED / ROAST POTATOES INCL HOMEMADE DISHES, value: 0.00032
x: BROWN GRANARY 

Proportion of HFSS based on the most granular categories (i.e., FoodName)

In [283]:
food_groups_values = {}
for x in food_noalcnovit['FoodName'].unique(): 
    value = np.sum((food_noalcnovit["FoodName"] == x)*(food_noalcnovit["lesshealthy"]))/np.sum(food_noalcnovit["FoodName"] == x)
    food_groups_values[x] = value   
    
food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]
    print (f'x: {x}, value: {value:.5f}')

x: BARLEY PEARL WHITE BOILED IN WATER, value: 0.00000
x: BARLEY WHOLE GRAIN DRIED, value: 0.00000
x: BARLEY WHOLEGRAIN BOILED IN WATER, value: 0.00000
x: WHEATGERM, value: 0.00000
x: BRAN WHEAT, value: 0.00000
x: CORNMEAL SIFTED DRY, value: 0.00000
x: CORN MEAL UNSIFTED DRIED, value: 0.00000
x: CUSTARD POWDER, value: 0.00000
x: FLOUR BROWN (85%), value: 0.00000
x: FLOUR CHAPATI BROWN, value: 0.00000
x: FLOUR CHAPATI WHITE, value: 0.00000
x: FLOUR WHITE SELF RAISING, value: 0.00000
x: FLOUR RICE, value: 0.00000
x: RYE FLOUR (100%), value: 0.00000
x: SOYA FLOUR, value: 0.00000
x: FLOUR WHITE BREADMAKING, value: 0.00000
x: FLOUR WHOLEMEAL (100%), value: 0.00000
x: OATMEAL RAW, value: 0.00000
x: OATS, ROLLED, PLAIN, DRY WEIGHT, NOT QUAKER, READY BREK/ INSTANT OATS, value: 0.00000
x: DO NOT USE- OATS, ROLLED, PLAIN, DRY WEIGHT, NOT QUAKER, READY BREK/ INSTANT OATS, value: 0.00000
x: PASTA MACARONI BOILED, value: 0.00000
x: PASTA NOODLES BOILED, value: 0.00000
x: PASTA NOODLES EGG BOILED, va

Proportion of UPF only based on more granular categories (i.e., SubFoodGroupDesc)

In [284]:
food_groups_values = {}
for x in food_noalcnovit['SubFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["SubFoodGroupDesc"] == x)*(food_noalcnovit["NOVA_UPF_NUPF_boolean"])*(~food_noalcnovit["lesshealthy"]))/(np.sum(food_noalcnovit["SubFoodGroupDesc"] == x))
    food_groups_values[x] = value  

food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]
    print (f'x: {x}, value: {value:.5f}')

x: CEREAL BASED MILK PUDDINGS - HOMEMADE, value: 0.00000
x: BUNS CAKES & PASTRIES HOMEMADE, value: 0.00000
x: BISCUITS HOMEMADE, value: 0.00000
x: DAIRY DESSERTS HOMEMADE, value: 0.00000
x: FRUIT PIES MANUFACTURED, value: 0.00000
x: SPONGE PUDDINGS - HOMEMADE, value: 0.00000
x: SPONGE PUDDINGS - MANUFACTURED, value: 0.00000
x: WHOLE MILK, value: 0.00000
x: SEMI SKIMMED MILK, value: 0.00000
x: CHEDDAR CHEESE, value: 0.00000
x: COTTAGE CHEESE, value: 0.00000
x: BUTTER, value: 0.00000
x: OTHER COOKING FATS AND OILS NOT PUFA, value: 0.00000
x: BLOCK MARGARINE, value: 0.00000
x: SOFT MARGARINE NOT POLYUNSATURATED, value: 0.00000
x: POLYUNSATURATED MARGARINE, value: 0.00000
x: OTHER LAMB INCLUDING HOMEMADE RECIPE DISHES, value: 0.00000
x: OTHER OILY FISH INCLUDING HOMEMADE DISHES, value: 0.00000
x: CANNED FRUIT IN JUICE, value: 0.00000
x: LEAFY GREEN VEGETABLES NOT RAW, value: 0.00000
x: CARROTS NOT RAW, value: 0.00000
x: PEAS NOT RAW, value: 0.00000
x: TOMATOES RAW, value: 0.00000
x: APPLES

Proportion of HFSS only based on more granular categories (i.e., SubFoodGroupDesc)

In [285]:
food_groups_values = {}
for x in food_noalcnovit['SubFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["SubFoodGroupDesc"] == x)*(food_noalcnovit["lesshealthy"])*(~food_noalcnovit["NOVA_UPF_NUPF_boolean"]))/(np.sum(food_noalcnovit["SubFoodGroupDesc"] == x))
    food_groups_values[x] = value  

food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]
    print (f'x: {x}, value: {value:.5f}')

x: PASTA MANUFACTURED PRODUCTS & READY MEALS, value: 0.00000
x: RICE MANUFACTURED PRODUCTS & READY MEALS, value: 0.00000
x: BROWN GRANARY AND WHEATGERM BREAD, value: 0.00000
x: BUNS CAKES & PASTRIES MANUFACTURED, value: 0.00000
x: OTHER BREAD, value: 0.00000
x: WHITE BREAD (NOT HIGH FIBRE; NOT MULTISEED BREAD), value: 0.00000
x: WHOLEMEAL BREAD, value: 0.00000
x: OTHER BREAKFAST CEREALS (NOT HIGH FIBRE), value: 0.00000
x: BISCUITS MANUFACTURED / RETAIL, value: 0.00000
x: CRISPS AND SAVOURY SNACKS, value: 0.00000
x: MANUFACTURED EGG PRODUCTS INCLUDING READY MEALS, value: 0.00000
x: OTHER CEREAL BASED PUDDINGS - MANUFACTURED, value: 0.00000
x: FRUIT PIES MANUFACTURED, value: 0.00000
x: CEREAL BASED MILK PUDDINGS - MANUFACTURED, value: 0.00000
x: SPONGE PUDDINGS - MANUFACTURED, value: 0.00000
x: ICE CREAM, value: 0.00000
x: SEMI SKIMMED MILK, value: 0.00000
x: SKIMMED MILK, value: 0.00000
x: NUTRITION POWDERS AND DRINKS, value: 0.00000
x: COTTAGE CHEESE, value: 0.00000
x: BLOCK MARGARINE,

Proportion of UPF based on the most granular categories (i.e., FoodName)

In [286]:
food_groups_values = {}
for x in food_noalcnovit['FoodName'].unique(): 
    value = np.sum((food_noalcnovit["FoodName"] == x)*(food_noalcnovit["NOVA_UPF_NUPF_boolean"]))/(np.sum(food_noalcnovit["FoodName"] == x))
    food_groups_values[x] = value  

food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]
    print (f'x: {x}, value: {value:.5f}')

x: BARLEY PEARL WHITE BOILED IN WATER, value: 0.00000
x: BARLEY WHOLE GRAIN DRIED, value: 0.00000
x: BARLEY WHOLEGRAIN BOILED IN WATER, value: 0.00000
x: WHEATGERM, value: 0.00000
x: BRAN WHEAT, value: 0.00000
x: CORNFLOUR, value: 0.00000
x: CORNMEAL SIFTED DRY, value: 0.00000
x: CORN MEAL UNSIFTED DRIED, value: 0.00000
x: FLOUR BROWN (85%), value: 0.00000
x: FLOUR CHAPATI BROWN, value: 0.00000
x: FLOUR CHAPATI WHITE, value: 0.00000
x: FLOUR WHITE SELF RAISING, value: 0.00000
x: FLOUR RICE, value: 0.00000
x: RYE FLOUR (100%), value: 0.00000
x: SOYA FLOUR, value: 0.00000
x: FLOUR WHITE BREADMAKING, value: 0.00000
x: FLOUR WHOLEMEAL (100%), value: 0.00000
x: OATMEAL RAW, value: 0.00000
x: OATS, ROLLED, PLAIN, DRY WEIGHT, NOT QUAKER, READY BREK/ INSTANT OATS, value: 0.00000
x: DO NOT USE- OATS, ROLLED, PLAIN, DRY WEIGHT, NOT QUAKER, READY BREK/ INSTANT OATS, value: 0.00000
x: PASTA MACARONI BOILED, value: 0.00000
x: PASTA NOODLES BOILED, value: 0.00000
x: PASTA NOODLES EGG BOILED, value: 

Proportion of upf only based on more granular categories (i.e., SubFoodGroupDesc)

In [287]:
food_groups_values = {}
for x in food_noalcnovit['SubFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["SubFoodGroupDesc"] == x)*(food_noalcnovit["NOVA_UPF_NUPF_boolean"])*~(food_noalcnovit["lesshealthy"]))/np.sum(food_noalcnovit["SubFoodGroupDesc"] == x)
    food_groups_values[x] = value 
    
food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]    
    print (f'x: {x}, value: {value:.5f}')

x: CEREAL BASED MILK PUDDINGS - HOMEMADE, value: 0.00000
x: BUNS CAKES & PASTRIES HOMEMADE, value: 0.00000
x: BISCUITS HOMEMADE, value: 0.00000
x: DAIRY DESSERTS HOMEMADE, value: 0.00000
x: FRUIT PIES MANUFACTURED, value: 0.00000
x: SPONGE PUDDINGS - HOMEMADE, value: 0.00000
x: SPONGE PUDDINGS - MANUFACTURED, value: 0.00000
x: WHOLE MILK, value: 0.00000
x: SEMI SKIMMED MILK, value: 0.00000
x: CHEDDAR CHEESE, value: 0.00000
x: COTTAGE CHEESE, value: 0.00000
x: BUTTER, value: 0.00000
x: OTHER COOKING FATS AND OILS NOT PUFA, value: 0.00000
x: BLOCK MARGARINE, value: 0.00000
x: SOFT MARGARINE NOT POLYUNSATURATED, value: 0.00000
x: POLYUNSATURATED MARGARINE, value: 0.00000
x: OTHER LAMB INCLUDING HOMEMADE RECIPE DISHES, value: 0.00000
x: OTHER OILY FISH INCLUDING HOMEMADE DISHES, value: 0.00000
x: CANNED FRUIT IN JUICE, value: 0.00000
x: LEAFY GREEN VEGETABLES NOT RAW, value: 0.00000
x: CARROTS NOT RAW, value: 0.00000
x: PEAS NOT RAW, value: 0.00000
x: TOMATOES RAW, value: 0.00000
x: APPLES

Overlap between UPF and HFSS across more granular food groups (SubFoodGroupDesc)

In [288]:
food_groups_values = {}
for x in food_noalcnovit['SubFoodGroupDesc'].unique(): 
    value = np.sum((food_noalcnovit["SubFoodGroupDesc"] == x)*(food_noalcnovit["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit["lesshealthy"]))/np.sum(food_noalcnovit["SubFoodGroupDesc"] == x)
    food_groups_values[x] = value 
    
food_groups_values_sorted = {x: value for x, value in sorted(food_groups_values.items(), key=lambda item: item[1])}    
for x in food_groups_values_sorted:
    value = food_groups_values_sorted[x]    
    print (f'x: {x}, value: {value:.5f}')

x: OTHER RICE INCLUDING HOMEMADE DISHES, value: 0.00000
x: BEANS AND PULSES INCL READY MEAL & HOMEMADE DISHES, value: 0.00000
x: OTHER PASTA INCLUDING HOMEMADE DISHES, value: 0.00000
x: CEREAL BASED MILK PUDDINGS - HOMEMADE, value: 0.00000
x: WHOLE MILK, value: 0.00000
x: SEMI SKIMMED MILK, value: 0.00000
x: SKIMMED MILK, value: 0.00000
x: CHEDDAR CHEESE, value: 0.00000
x: COTTAGE CHEESE, value: 0.00000
x: OTHER COOKING FATS AND OILS NOT PUFA, value: 0.00000
x: POLYUNSATURATED OILS, value: 0.00000
x: OTHER BEEF & VEAL INCLUDING HOMEMADE RECIPE DISHES, value: 0.00000
x: OTHER CHICKEN / TURKEY INCL HOMEMADE RECIPE DISHES, value: 0.00000
x: MANUFACTURED LAMB PRODUCTS INCLUDING READY MEALS, value: 0.00000
x: BAKED BEANS, value: 0.00000
x: OTHER WHITE FISH INCLUDING HOMEMADE DISHES, value: 0.00000
x: OTHER OILY FISH INCLUDING HOMEMADE DISHES, value: 0.00000
x: OTHER CANNED TUNA INCLUDING HOMEMADE DISHES, value: 0.00000
x: OTHER SHELLFISH INCLUDING HOMEMADE DISHES, value: 0.00000
x: CANNED F

Sub-group analyses: robustness check

Analyses for males

In [289]:
food_noalcnovit_male = food_noalcnovit[food_noalcnovit['Sex']==1]

Proportion of HFSS foods 

In [290]:
np.sum(food_noalcnovit_male["lesshealthy"])/len(food_noalcnovit_male["lesshealthy"])

0.34830083837380915

Proportion of UPF foods

In [291]:
np.sum(food_noalcnovit_male["NOVA_UPF_NUPF_boolean"])/len(food_noalcnovit_male["NOVA_UPF_NUPF_boolean"])

0.3817728516911979

Proportion of UPFs and HFSS among UPF sample

In [292]:
np.sum((food_noalcnovit_male["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit_male["lesshealthy"]))/len(food_noalcnovit_male[food_noalcnovit_male["NOVA_UPF_NUPF_boolean"]])

0.556112939149861

The proportion of food and drink items from NDNS that are: neither HFSS (NPM 2004/5) nor UPFs; UPFs only;
HFSS only (NPM 2004/5); HFSS (NPM 2004/5) and UPFs based on the overall sample.
For the summary graphs aggregate for food and drinks

In [293]:
value_notupf_healthy_m = np.sum((~food_noalcnovit_male["NOVA_UPF_NUPF_boolean"])*~food_noalcnovit_male["lesshealthy"])/len(food_noalcnovit_male)
value_upf_healthy_m = np.sum((food_noalcnovit_male["NOVA_UPF_NUPF_boolean"])*~food_noalcnovit_male["lesshealthy"])/len(food_noalcnovit_male)
value_notupf_lesshealthy_m = np.sum((~food_noalcnovit_male["NOVA_UPF_NUPF_boolean"])*food_noalcnovit_male["lesshealthy"])/len(food_noalcnovit_male)
value_upf_lesshealthy_m = np.sum((food_noalcnovit_male["NOVA_UPF_NUPF_boolean"])*food_noalcnovit_male["lesshealthy"])/len(food_noalcnovit_male)
print (value_notupf_healthy_m)
print (value_upf_healthy_m)
print (value_notupf_lesshealthy_m)
print (value_upf_lesshealthy_m)

0.48223513257660905
0.16946402904958183
0.13599201573219308
0.21230882264161607


For females

In [294]:
food_noalcnovit_female = food_noalcnovit[food_noalcnovit['Sex']==2]

Proportion of HFSS foods

In [295]:
np.sum(food_noalcnovit_female["lesshealthy"])/len(food_noalcnovit_female["lesshealthy"])

0.32247823989861574

Proportion of UPFs

In [296]:
np.sum(food_noalcnovit_female["NOVA_UPF_NUPF_boolean"])/len(food_noalcnovit_female["NOVA_UPF_NUPF_boolean"])

0.34601267093094246

Proportion of UPFs and HFSS among UPF sample

In [297]:
np.sum((food_noalcnovit_female["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit_female["lesshealthy"]))/len(food_noalcnovit_female[food_noalcnovit_female["NOVA_UPF_NUPF_boolean"]])

0.5561946701085718

The proportion of food and drink items from NDNS that are: neither HFSS (NPM 2004/5) nor UPFs; UPFs only;
HFSS only (NPM 2004/5); HFSS (NPM 2004/5) and UPFs based on the overall sample.
For the summary graphs aggregate for food and drinks

In [298]:
value_notupf_healthy_f = np.sum((~food_noalcnovit_female["NOVA_UPF_NUPF_boolean"])*~food_noalcnovit_female["lesshealthy"])/len(food_noalcnovit_female)
value_upf_healthy_f = np.sum((food_noalcnovit_female["NOVA_UPF_NUPF_boolean"])*~food_noalcnovit_female["lesshealthy"])/len(food_noalcnovit_female)
value_notupf_lesshealthy_f = np.sum((~food_noalcnovit_female["NOVA_UPF_NUPF_boolean"])*food_noalcnovit_female["lesshealthy"])/len(food_noalcnovit_female)
value_upf_lesshealthy_f = np.sum((food_noalcnovit_female["NOVA_UPF_NUPF_boolean"])*food_noalcnovit_female["lesshealthy"])/len(food_noalcnovit_female)
print (value_notupf_healthy_f)
print (value_upf_healthy_f)
print (value_notupf_lesshealthy_f)
print (value_upf_lesshealthy_f)

0.5239594925322631
0.15356226756912109
0.13002783653679437
0.1924504033618214


By energy for males

Proportion of energy derived from HFSS 

In [299]:
np.sum((food_noalcnovit_male["lesshealthy"])*(food_noalcnovit_male['Energykcal']))/np.sum(food_noalcnovit_male["Energykcal"])

0.4786796692025585

Proportion of energy derived from UPFs 

In [300]:
np.sum((food_noalcnovit_male["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit_male['Energykcal']))/np.sum(food_noalcnovit_male["Energykcal"])

0.6098841954917376

create A UPF subsample

In [301]:
food_noalcnovit_upf_male = food_noalcnovit_male[food_noalcnovit_male['NOVA_UPF_NUPF_boolean']]

Proportion of energy derived from HFSS based on UPF sample

In [302]:
np.sum((food_noalcnovit_upf_male["lesshealthy"])*(food_noalcnovit_upf_male['Energykcal']))/np.sum (food_noalcnovit_upf_male["Energykcal"])

0.5848493803402596

The proportion of food and drink items from NDNS that are: neither HFSS (NPM 2004/5) nor UPFs; UPFs only;
HFSS only (NPM 2004/5); HFSS (NPM 2004/5) and UPFs based on the overall sample.
For the summary graphs aggregate for food and drinks

In [303]:
value_notupf_healthy = np.sum(((~food_noalcnovit_male["NOVA_UPF_NUPF_boolean"]))*(~food_noalcnovit_male["lesshealthy"])*(food_noalcnovit_male['Energykcal']))/np.sum(food_noalcnovit_male['Energykcal'])
value_upf_healthy = np.sum(((food_noalcnovit_male["NOVA_UPF_NUPF_boolean"]))*(~food_noalcnovit_male["lesshealthy"])*(food_noalcnovit_male['Energykcal']))/np.sum(food_noalcnovit_male['Energykcal'])
value_notupf_lesshealthy = np.sum(((~food_noalcnovit_male["NOVA_UPF_NUPF_boolean"]))*((food_noalcnovit_male["lesshealthy"]))*(food_noalcnovit_male['Energykcal']))/np.sum(food_noalcnovit_male['Energykcal'])
value_upf_lesshealthy = np.sum(((food_noalcnovit_male["NOVA_UPF_NUPF_boolean"]))*(food_noalcnovit_male["lesshealthy"])*(food_noalcnovit_male['Energykcal']))/np.sum(food_noalcnovit_male['Energykcal'])
print (value_notupf_healthy)
print (value_upf_healthy)
print (value_notupf_lesshealthy)
print (value_upf_lesshealthy)

0.2681265291183646
0.25319380167907724
0.1219892753898978
0.35669039381266066


By energy for females

Proportion of energy derived from HFSS 

In [304]:
np.sum((food_noalcnovit_female["lesshealthy"])*(food_noalcnovit_female['Energykcal']))/np.sum(food_noalcnovit_female["Energykcal"])

0.4686852842799541

Proportion of energy derived from UPFs 

In [305]:
np.sum((food_noalcnovit_female["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit_female['Energykcal']))/np.sum(food_noalcnovit_female["Energykcal"])

0.5860593713462839

Create a UPF sample only

In [306]:
food_noalcnovit_upf_female = food_noalcnovit_female[food_noalcnovit_female['NOVA_UPF_NUPF_boolean']]

Overlap between UPFs and HFSS weighted by energy, based on the UPF sample

In [307]:
np.sum((food_noalcnovit_upf_female["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit_upf_female["lesshealthy"])*(food_noalcnovit_upf_female['Energykcal']))/np.sum (food_noalcnovit_upf_female["Energykcal"])

0.5888727319835516

The proportion of food and drink items from NDNS that are: neither HFSS (NPM 2004/5) nor UPFs; UPFs only;
HFSS only (NPM 2004/5); HFSS (NPM 2004/5) and UPFs based on the overall sample.
For the summary graphs aggregate for food and drinks

In [308]:
value_notupf_healthy = np.sum(((~food_noalcnovit_female["NOVA_UPF_NUPF_boolean"]))*(~food_noalcnovit_female["lesshealthy"])*(food_noalcnovit_female['Energykcal']))/np.sum(food_noalcnovit_female['Energykcal'])
value_upf_healthy = np.sum(((food_noalcnovit_female["NOVA_UPF_NUPF_boolean"]))*(~food_noalcnovit_female["lesshealthy"])*(food_noalcnovit_female['Energykcal']))/np.sum(food_noalcnovit_female['Energykcal'])
value_notupf_lesshealthy = np.sum(((~food_noalcnovit_female["NOVA_UPF_NUPF_boolean"]))*((food_noalcnovit_female["lesshealthy"]))*(food_noalcnovit_female['Energykcal']))/np.sum(food_noalcnovit_female['Energykcal'])
value_upf_lesshealthy = np.sum(((food_noalcnovit_female["NOVA_UPF_NUPF_boolean"]))*(food_noalcnovit_female["lesshealthy"])*(food_noalcnovit_female['Energykcal']))/np.sum(food_noalcnovit_female['Energykcal'])
print (value_notupf_healthy)
print (value_upf_healthy)
print (value_notupf_lesshealthy)
print (value_upf_lesshealthy)

0.290369727483011
0.2409449882370351
0.12357090117070524
0.3451143831092489


By weight for males

Proportion of weight from HFSS

In [309]:
np.sum((food_noalcnovit_male["lesshealthy"])*(food_noalcnovit_male['TotalGrams']))/np.sum(food_noalcnovit_male["TotalGrams"])

0.17828106252099474

Proportion of weight from UPFs

In [310]:
np.sum((food_noalcnovit_male["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit_male['TotalGrams']))/np.sum(food_noalcnovit_male["TotalGrams"])

0.3556992454939843

Sub-group analyses for UPF only

In [311]:
food_noalcnovit_upf_male = food_noalcnovit_male[food_noalcnovit_male['NOVA_UPF_NUPF_boolean']]

Overlap between UPFs and HFSS weighted 

In [312]:
np.sum((food_noalcnovit_upf_male["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit_upf_male["lesshealthy"])*(food_noalcnovit_upf_male['TotalGrams']))/np.sum (food_noalcnovit_upf_male["TotalGrams"])

0.3951467037611594

The proportion of food and drink items from NDNS that are: neither HFSS (NPM 2004/5) nor UPFs; UPFs only;
HFSS only (NPM 2004/5); HFSS (NPM 2004/5) and UPFs based on the overall sample.
For the summary graphs aggregate for food and drinks

In [313]:
value_notupf_healthy = np.sum(((~food_noalcnovit_male["NOVA_UPF_NUPF_boolean"]))*(~food_noalcnovit_male["lesshealthy"])*(food_noalcnovit_male['TotalGrams']))/np.sum(food_noalcnovit_male['TotalGrams'])
value_upf_healthy = np.sum(((food_noalcnovit_male["NOVA_UPF_NUPF_boolean"]))*(~food_noalcnovit_male["lesshealthy"])*(food_noalcnovit_male['TotalGrams']))/np.sum(food_noalcnovit_male['TotalGrams'])
value_notupf_lesshealthy = np.sum(((~food_noalcnovit_male["NOVA_UPF_NUPF_boolean"]))*((food_noalcnovit_male["lesshealthy"]))*(food_noalcnovit_male['TotalGrams']))/np.sum(food_noalcnovit_male['TotalGrams'])
value_upf_lesshealthy = np.sum(((food_noalcnovit_male["NOVA_UPF_NUPF_boolean"]))*(food_noalcnovit_male["lesshealthy"])*(food_noalcnovit_male['TotalGrams']))/np.sum(food_noalcnovit_male['TotalGrams'])
print (value_notupf_healthy)
print (value_upf_healthy)
print (value_notupf_lesshealthy)
print (value_upf_lesshealthy)

0.6065730763723001
0.21514586110670492
0.03772767813371533
0.14055338438727935


By weight for females

Proportion of weight derived from HFSS

In [314]:
np.sum((food_noalcnovit_female["lesshealthy"])*(food_noalcnovit_female['TotalGrams']))/np.sum(food_noalcnovit_female["TotalGrams"])

0.14402630468852576

Proportion of weight derived from UPFs

In [315]:
np.sum((food_noalcnovit_female["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit_female['TotalGrams']))/np.sum(food_noalcnovit_female["TotalGrams"])

0.30491023486342345

Create a UPF sample only

In [316]:
food_noalcnovit_upf_female = food_noalcnovit_female[food_noalcnovit_female['NOVA_UPF_NUPF_boolean']]

Overlap between UPF anf HFSS by weight

In [317]:
np.sum((food_noalcnovit_upf_female["NOVA_UPF_NUPF_boolean"])*(food_noalcnovit_upf_female["lesshealthy"])*(food_noalcnovit_upf_female['TotalGrams']))/np.sum (food_noalcnovit_upf_female["TotalGrams"])

0.37153430245215113

The proportion of food and drink items from NDNS that are: neither HFSS (NPM 2004/5) nor UPFs; UPFs only;
HFSS only (NPM 2004/5); HFSS (NPM 2004/5) and UPFs based on the overall sample.
For the summary graphs aggregate for food and drinks

In [318]:
value_notupf_healthy = np.sum(((~food_noalcnovit_female["NOVA_UPF_NUPF_boolean"]))*(~food_noalcnovit_female["lesshealthy"])*(food_noalcnovit_female['TotalGrams']))/np.sum(food_noalcnovit_female['TotalGrams'])
value_upf_healthy = np.sum(((food_noalcnovit_female["NOVA_UPF_NUPF_boolean"]))*(~food_noalcnovit_female["lesshealthy"])*(food_noalcnovit_female['TotalGrams']))/np.sum(food_noalcnovit_female['TotalGrams'])
value_notupf_lesshealthy = np.sum(((~food_noalcnovit_female["NOVA_UPF_NUPF_boolean"]))*((food_noalcnovit_female["lesshealthy"]))*(food_noalcnovit_female['TotalGrams']))/np.sum(food_noalcnovit_male['TotalGrams'])
value_upf_lesshealthy = np.sum(((food_noalcnovit_female["NOVA_UPF_NUPF_boolean"]))*(food_noalcnovit_female["lesshealthy"])*(food_noalcnovit_female['TotalGrams']))/np.sum(food_noalcnovit_female['TotalGrams'])
print (value_notupf_healthy)
print (value_upf_healthy)
print (value_notupf_lesshealthy)
print (value_upf_lesshealthy)

0.6643480718685547
0.19162562344291986
0.035186144745539355
0.11328461142050361
